# Chapter 13 处理关系数据

## 13.1 简介
只涉及一张数据表的数据分析是非常罕见的。通常来说，你会有很多个数据表，而且必须综合使用它们才能回答你所感兴趣的问题。存在于多个表中的这种数据统称为关系数据。本章我们介绍了三类操作来处理这种关系数据。

* 合并连接：向数据框中加入新变量，新变量的值是另一个数据框中的匹配观测。
* 筛选连接：根据是否匹配另一个数据框中的观测，筛选数据框中的观测。
* 集合操作：将观测作为集合元素来处理。 

## 13.2 准备工作
我们使用merge函数来研究一下nycflights中的关系数据。

In [1]:
import pandas as pd
# 导入数据
file=['airlines','airports','flights','planes','weather']
for filename in file:  
    f=open('nycflights/%s.csv'%filename)
    globals()[filename]=pd.read_csv(f)

In [2]:
# airlines：可以根据航空公司的缩写码查到公司全名。
airlines.head(5)

,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.
3,B6,JetBlue Airways
4,DL,Delta Air Lines Inc.


In [3]:
# airports：给出了每个机场的信息，通过faa机场编码进行标识。
airports.head(5)

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York


In [4]:
# planes：给出了每架飞机的信息，通过tailnum 进行标识。
planes.head(5)

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan


In [5]:
# weather：给出了纽约机场每小时的天气状况。
weather.head(5)

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2013,1,1,1,39.02,26.06,59.37,270.0,10.35702,NaN,0.0,1012.0,10.0,2013-1-1 1:00
1,EWR,2013,1,1,2,39.02,26.96,61.63,250.0,8.05546,NaN,0.0,1012.3,10.0,2013-1-1 2:00
2,EWR,2013,1,1,3,39.02,28.04,64.43,240.0,11.50780,NaN,0.0,1012.5,10.0,2013-1-1 3:00
3,EWR,2013,1,1,4,39.92,28.04,62.21,250.0,12.65858,NaN,0.0,1012.2,10.0,2013-1-1 4:00
4,EWR,2013,1,1,5,39.02,28.04,64.43,260.0,12.65858,NaN,0.0,1011.9,10.0,2013-1-1 5:00


![title](img/数据关系图.png)

* flights与planes通过单变量tailnum相连；
* flights与airlines通过变量carrier相连；
* flights与airports通过两种方式相连（变量origin 和dest）；
* flights与weather通过变量origin（位置）以及year、month、day 和hour（时间）相连。

## 13.3 键
* 键是能唯一标识观测的变量（或变量集合），可以用于连接每对数据表。简单情况下，单个变量就足以标识一个观测。例如，每架飞机都可以由tailnum唯一标识。其他情况可能需要多个变量。例如，要想标识weather中的观测，你需要5个变量：year、month、day、hour 和origin。
* 键的类型有两种。主键是唯一标识其所在数据表中的观测。外键是唯一标识另一个数据表中的观测，是用来和其他表建立联系用的。这种关系通常是一对多的。
* 一旦识别出表的主键，最好验证一下，看看它们能否真正唯一标识每个观测。

In [6]:
planes[planes.duplicated(keep=False, subset='tailnum')]

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine


In [7]:
weather[weather.duplicated(keep=False, subset=['year', 'month', 'day', 'hour', 'origin'])]

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
7318,EWR,2013,11,3,1,51.98,39.02,61.15,310.0,6.90468,NaN,0.0,1009.8,10.0,2013-11-3 1:00
7319,EWR,2013,11,3,1,50.00,39.02,65.80,290.0,5.75390,NaN,0.0,1010.5,10.0,2013-11-3 1:00
16023,JFK,2013,11,3,1,53.96,37.94,54.51,320.0,9.20624,NaN,0.0,1009.8,10.0,2013-11-3 1:00
16024,JFK,2013,11,3,1,51.98,37.94,58.62,310.0,6.90468,NaN,0.0,1010.5,10.0,2013-11-3 1:00
24729,LGA,2013,11,3,1,55.04,39.02,54.67,330.0,9.20624,NaN,0.0,1009.3,10.0,2013-11-3 1:00
24730,LGA,2013,11,3,1,53.96,39.92,58.89,310.0,8.05546,NaN,0.0,1010.2,10.0,2013-11-3 1:00


* 如果一张表没有主键，有时就需要使用reset_index()函数为表加上一个主键。这样一来，如果你完成了一些筛选工作，并想要使用原始数据检查的话，就可以更容易地匹配观测。这种主键称为代理键。

In [9]:
flights.reset_index()

,index,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-1-1 5:00
1,1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-1-1 5:00
2,2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-1-1 5:00
3,3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-1-1 5:00
4,4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-1-1 6:00
5,5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-1-1 5:00
6,6,2013,1,1,555.0,600,-5.0,913.0,854,19.0,B6,507,N516JB,EWR,FLL,158.0,1065,6,0,2013-1-1 6:00
7,7,2013,1,1,557.0,600,-3.0,709.0,723,-14.0,EV,5708,N829AS,LGA,IAD,53.0,229,6,0,2013-1-1 6:00
8,8,2013,1,1,557.0,600,-3.0,838.0,846,-8.0,B6,79,N593JB,JFK,MCO,140.0,944,6,0,2013-1-1 6:00
9,9,2013,1,1,558.0,600,-2.0,753.0,745,8.0,AA,301,N3ALAA,LGA,ORD,138.0,733,6,0,2013-1-1 6:00


## 13.4 合并连接
合并连接可以将两个表格中的变量组合起来，它先通过两个表格的键匹配观测，然后将一个表格中的变量复制到另一个表格中。

pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True)

参数如下：
* left: 拼接的左侧DataFrame对象。
* right: 拼接的右侧DataFrame对象。
* on: 要加入的列或索引级别名称。必须在左侧和右侧DataFrame对象中找到。如果未传递且left_index和right_index为False，则DataFrame中的列的交集将被推断为连接键。
* left_on:左侧DataFrame中的列或索引级别用作键。可以是列名，索引级名称，也可以是长度等于DataFrame长度的数组。
* right_on: 左侧DataFrame中的列或索引级别用作键。可以是列名，索引级名称，也可以是长度等于DataFrame长度的数组。
* left_index: 如果为True，则使用左侧DataFrame中的索引（行标签）作为其连接键。对于具有MultiIndex（分层）的DataFrame，级别数必须与右侧DataFrame中的连接键数相匹配。
* right_index: 与left_index功能相似。
* how: 选择'left', 'right', 'outer', 'inner'其中之一。默认inner。inner是内连接，取交集，outer是全连接，取并集，left是左连接，左边取全部，右边取部分，没有值则用NaN填充，right是右连接，右边取全部，左边取部分，没有值则用NaN填充。
* sort: 按字典顺序通过连接键对结果DataFrame进行排序。 默认为True。
* suffixes: 用于重叠列的字符串后缀元组。 默认为（'_x', '_y'）。
* copy: 默认为True,总是将数据复制到数据结构中。

In [10]:
# 左连接组合flights和airlines数据框
data1 = pd.merge(flights[['year', 'month', 'day', 'hour', 'origin', 'carrier']], airlines, how='left', on='carrier', left_on=None, right_on=None,
         left_index=False, right_index=False)

data1.head(5)

,year,month,day,hour,origin,carrier,name
0,2013,1,1,5,EWR,UA,United Air Lines Inc.
1,2013,1,1,5,LGA,UA,United Air Lines Inc.
2,2013,1,1,5,JFK,AA,American Airlines Inc.
3,2013,1,1,5,JFK,B6,JetBlue Airways
4,2013,1,1,6,LGA,DL,Delta Air Lines Inc.


### 13.4.1 理解连接
我们可视化表示4 种合并连接：1 种内连接和3 种外连接。在处理实际数据时，键并不能总是唯一地标识观测，因此我们还讨论如何处理不能唯一匹配的情况。

### 13.4.2 内连接

只要两个观测的键是相等的，内连接就可以匹配。

![title](img/1.png)

In [12]:
x = pd.DataFrame({'x':['x1','x2','x3']}, index=[1,2,3])
y = pd.DataFrame({'y':['y1','y2','y3']}, index=[1,2,4])

# inner
pd.merge(x, y, how='inner', left_on=None, right_on=None,
         left_index=True, right_index=True)

,x,y
1,x1,y1
2,x2,y2


### 13.4.3 外连接

内连接保留同时存在于两个表中的观测，外连接则保留至少存在于一个表中的观测。
* 左连接：保留左侧对象中的所有观测。
* 右连接：保留右侧对象中的所有观测。
* 全连接：保留左侧对象和右侧对象中的所有观测。

如果某些键不存在配对，这些连接会向每个表中添加额外的“虚拟”观测，其值则用NA来填充。

![title](img/2.png)

In [13]:
# 举outer为例
pd.merge(x, y, how='outer', left_on=None, right_on=None,
         left_index=True, right_index=True)

,x,y
1,x1,y1
2,x2,y2
3,x3,NaN
4,NaN,y3


### 13.4.4 重复键
当键不唯一时将会发生的两种情况：
* 一张表中具有重复键。通常来说，当存在一对多关系时，如果你想要向表中添加额外信息，就会出现这种情况。
* 两张表中都有重复键。这通常意味着出现了错误，因为键在任意一张表中都不能唯一标识观测。当连接这样的重复键时，你会得到所有可能的组合，即笛卡儿积：

![title](img/3.png)

In [14]:
# 一张表中具有重复键
x = pd.DataFrame({'x':['x1','x2','x3','x4']}, index=[1,2,2,1])
y = pd.DataFrame({'y':['y1','y2']}, index=[1,2])

# left
pd.merge(x, y, how='left', left_on=None, right_on=None,
         left_index=True, right_index=True)

,x,y
1,x1,y1
1,x4,y1
2,x2,y2
2,x3,y2


![title](img/4.png)

In [15]:
# 两张表中都有重复键
x = pd.DataFrame({'x':['x1','x2','x3','x4']}, index=[1,2,2,3])
y = pd.DataFrame({'y':['y1','y2','y3','y4']}, index=[1,2,2,3])

# left
pd.merge(x, y, how='left', left_on=None, right_on=None,
         left_index=True, right_index=True)

,x,y
1,x1,y1
2,x2,y2
2,x2,y3
2,x3,y2
2,x3,y3
3,x4,y4


### 13.4.5 定义键列
* on缺省。这会使用存在于两个表中的所有变量，这种方式称为自然连接。

In [16]:
data2 = pd.merge(flights[['year', 'month', 'day', 'hour', 'origin', 'carrier']], airlines, how='left', left_on=None, right_on=None,
         left_index=False, right_index=False)
data2.head(5)

,year,month,day,hour,origin,carrier,name
0,2013,1,1,5,EWR,UA,United Air Lines Inc.
1,2013,1,1,5,LGA,UA,United Air Lines Inc.
2,2013,1,1,5,JFK,AA,American Airlines Inc.
3,2013,1,1,5,JFK,B6,JetBlue Airways
4,2013,1,1,6,LGA,DL,Delta Air Lines Inc.


* 字符向量on = "x"，或left_on = "x", right_on = "y"。这种方式与自然连接很相似，但只使用某些公共变量。

In [17]:
data3 = pd.merge(flights[['year', 'month', 'day', 'hour', 'origin', 'dest', 'tailnum', 'carrier']], airports, how='left', left_on='dest', right_on='faa',
         left_index=False, right_index=False)
data3.head(5)

,year,month,day,hour,origin,dest,tailnum,carrier,faa,name,lat,lon,alt,tz,dst,tzone
0,2013,1,1,5,EWR,IAH,N14228,UA,IAH,George Bush Intercontinental,29.984433,-95.341442,97.0,-6.0,A,America/Chicago
1,2013,1,1,5,LGA,IAH,N24211,UA,IAH,George Bush Intercontinental,29.984433,-95.341442,97.0,-6.0,A,America/Chicago
2,2013,1,1,5,JFK,MIA,N619AA,AA,MIA,Miami Intl,25.793250,-80.290556,8.0,-5.0,A,America/New_York
3,2013,1,1,5,JFK,BQN,N804JB,B6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,1,1,6,LGA,ATL,N668DN,DL,ATL,Hartsfield Jackson Atlanta Intl,33.636719,-84.428067,1026.0,-5.0,A,America/New_York


## 13.5 筛选连接
筛选连接匹配观测的方式与合并连接相同，但前者会影响观测。如果有重复键时，筛选连接不会像合并连接那样造成重复的行。
* 半连接保留左表中与右表中的观测相匹配的所有观测。可以像合并连接一样连接两个表，但不添加新列。
* 全连接丢弃左表中与右表中的观测相匹配的所有观测。即保留左表中那些没有匹配右表中的行。

![title](img/5.png)

In [29]:
#半连接
x = pd.DataFrame({'x':['x1','x2','x3']}, index=[1,2,3])
y = pd.DataFrame({'y':['y1','y2','y3']}, index=[1,2,4])
x.loc[[a in y.index for a in x.index],:]

,x
1,x1
2,x2


![title](img/6.png)

In [18]:
#半连接，筛选连接不会像合并连接那样造成重复的行
x = pd.DataFrame({'x':['x1','x2','x3','x4']}, index=[1,2,2,3])
y = pd.DataFrame({'y':['y1','y2','y3','y4']}, index=[1,2,2,3])
x.loc[[a in y.index for a in x.index],:]

,x
1,x1
2,x2
2,x3
3,x4


![title](img/7.png)

In [21]:
#外连接
x.loc[[not(a in y.index) for a in x.index],:]

,x
3,x3


## 13.7 集合操作
所有集合操作都是作用于整行的，需要左右表具有相同的变量，并将观测按照集合来处理。
* intersect: 返回既在左表，又在右表中的观测。
* union: 返回左表或右表中的唯一观测。
* setdiff: 返回在左表，但不在右表中的观测。

In [35]:
x = pd.DataFrame({'x':[1,2], 'y':[1,1]})
x

,x,y
0,1,1
1,2,1


In [34]:
y = pd.DataFrame({'x':[1,1], 'y':[1,2]})
y

,x,y
0,1,1
1,1,2


In [33]:
# intersect
intersect = pd.merge(x, y, how='inner')
intersect

,x,y
0,1,1


In [24]:
# union
pd.merge(x, y, how='outer')

,x,y
0,1,1
1,2,1
2,1,2


In [27]:
# setdiff1
pd.concat([intersect, x]).drop_duplicates(keep=False).reset_index(drop=True)

,x,y
0,2,1


In [28]:
# setdiff2
pd.concat([intersect, y]).drop_duplicates(keep=False).reset_index(drop=True)

,x,y
0,1,2


其他处理关系数据的函数：join()函数，concat()函数等